In [ ]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_state_qsphere
from qiskit.quantum_info import Operator
# visualize eigenstate
def to_bloch_vector(ucirc, statevector):
new_state = tatevector.evolve(ucirc)
bloch_vector = [
new_state.expectation_value(Operator([[0, 1], [1, 0]])).real, # X
new_state.expectation_value(Operator([[0, -1j], [1j, 0]])).real, # Y
new_state.expectation_value(Operator([[1, 0], [0, -1]])).real # Z
]
return bloch_vector
# visualize eigenvalue
def plot_complex_nmb(phasors):
# Create a new figure
plt.figure(figsize=(4, 4))
# plot each phasor as a vector
for phasor in phasors:
real = np.real(phasor)
imag = np.imag(phasor)
plt.quiver(0, 0, real, imag, angles='xy', scale_units='xy', scale=1, color=’r’)
plt.text(real * 1.1, imag * 1.1, f’{phasor:.2f}’, fontsize=12)
# set up the plot axes
plt.xlim(-2, 2) # Adjust these limits based on your phasors
plt.ylim(-2, 2)
plt.axhline(0, color=’black’,linewidth=0.5)
plt.axvline(0, color=’black’,linewidth=0.5)
plt.xlabel(’Real’)
plt.ylabel(’Imaginary’)
plt.grid(True)
plt.gca().set_aspect(’equal’, adjustable=’box’)
plt.title(’Phasor Diagram’)
# show the plot
plt.show()
phi = np.pi/3
ucirc = QuantumCircuit(1,1)
ucirc.rx(phi,0)
state0 = Statevector([1,0])
state1 = Statevector([0,1])
plus = (state0+state1)/np.sqrt(2)
minus = (state0-state1)/np.sqrt(2)
bloch_0 = to_bloch_vector(ucirc, plus)
bloch_1 = to_bloch_vector(ucirc, minus)
# plot the Bloch vector
plot_bloch_vector([bloch_0, bloch_1]).savefig(’bloch.pdf’)
8
phasors = [np.exp(-0.5j*phi), np.exp(0.5j*phi)]
plot_complex_nmb(phasors)
# create quantum circuit
qc = QuantumCircuit(5, 4)
# prepare eigenstate
qc.h(4)
# step 1: add hadamard
for qubit in range(4):
qc.h(qubit)
qc.barrier()
# step 2: controlled unitary operations
for reg1_index in range(4):
repetitions = 2**reg1_index
for i in range(repetitions):
qc.crx(phi, reg1_index, 4)
qc.barrier()
# step 3: IQFT on the first register
iqft = QFT(num_qubits=4, inverse=True, insert_barriers=True)
qc.append(iqft, [0,1,2,3])
qc.barrier()
# step 4: measurement
for reg1_index in range(4):
qc.measure(reg1_index, reg1_index)
qc.draw(output=’mpl’).savefig(’qpe_4.pdf’)
sampler = Sampler()
job = sampler.run(circuits=qc, shots=5000)
result = job.result()
# get the result counts and convert quasi-probabilities to probabilities
counts = result.quasi_dists[0].binary_probabilities()
# print result
print("Measuremet result:", result.quasi_dists[0])
# plot the result as a histogram
plt.rc(’font’, family=’serif’, serif=’Times New Roman’)
plot_histogram(counts, figsize=(8,4.5)).savefig(’hist_4.pdf’)

SyntaxError: invalid character '’' (U+2019) (1110996763.py, line 28)